In [1]:
# Import necessary modules
import cvxpy as cvx
import numpy as np
import scipy
from scipy import random
from scipy import linalg
from scipy.linalg import eigh

### Steepest Descent and Projection Method

For a first order Steepest Descent method for the SNL problem, one of the main issues is ensuring that the semi-definite constraint is enforced at each iteration. In this project, we try 3 different approaches: 

 * Eigenvalue decomposition, ensuring that none of the eigenvalues are negative.
 * LDL decomposition, ensuring that none of the pivots in D are negative.
 * Eigenvalue decomposition, keeping only the k largest positive eigenvalues.

We find that the eigenvalue decomposition works consistently for ten sensors, four anchors, and two dimensions. For the LDL decomposition, we realized this approach is ill-suited to maintaining positive semi definitness. This is becase the LDL decomposition requires the matrix to be *positive definite* in the first place in order to run. This leads the algorithm to fail once the point falls out of the feasible region. For the approach using only k of the largest positive eigenvalues, we predictably succeed if k = n, as this simply becomes the first approach outlined. However, as soon as k < n, the algorithm never converges on the correct sensor locations. 

In [2]:
def generate_points(num, dim):
    return np.matrix(4 * np.random.random((num, dim)) - 2)

In [3]:
num_sensors = 10
num_anchors = 4
dim = 2
range_mult = 1

anchors = generate_points(num_anchors, dim) * range_mult
sensors = generate_points(num_sensors, dim)

d_sa = list(map(lambda s: list(map(lambda a: np.linalg.norm(a - s), anchors)), sensors))

d_ss = list(map(lambda s1: 
        list(map(lambda s2: np.linalg.norm(s1 - s2), sensors))
    , sensors))

In [4]:
# Same as from Question 1.
# Make first set of constraint matrices (look like identity)
def enforce_id(dim, num_sensors):
    matrices = []
    rhs = []
    for i in range(dim):
        new_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
        new_matrix[i,i] = 1
        matrices.append(new_matrix)
        rhs.append(1)
        
    return (matrices, rhs)

#Make second set of constraint matrices (symmetric holders) 
def enforce_id2(dim, num_sensors):
    matrices = []
    rhs = []
    for i in range(dim):
        for j in range(dim):
            new_matrix = np.identity(dim)
            if(j > i):
                new_matrix[i,j] = 1
                new_matrix[j,i] = 1
                big_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
                big_matrix[0:dim,0:dim] = new_matrix
                matrices.append(big_matrix)
                rhs.append(dim)
                
    return (matrices, rhs)


#Make third set of constraint matrices (anchors to sensors)
def sensor_constraints(dim, num_sensors):
    matrices = []
    rhs = []
    zero_vec_dim = np.zeros(dim)
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[i] = 1
            zero_vec_num_s[j] = -1
            
            new_vec = np.matrix(np.append(zero_vec_dim, zero_vec_num_s))
            
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            
            matrices.append(new_matrix)
            rhs.append(d_ss[i][j]**2)

    return (matrices, rhs)

#Make fourth set of constraint matrices (sensors to sensors)
def anchor_constraints(num_anchors, num_sensors):
    matrices = []
    rhs = []
    for i in range(num_anchors):
        for j in range(num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[j] = -1

            new_vec = np.append(np.array(anchors[i,:]), np.array(zero_vec_num_s))
            new_vec = np.matrix(new_vec)
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            matrices.append(new_matrix)
            rhs.append(d_sa[j][i]**2)

    return (matrices, rhs)

A = enforce_id(dim, num_sensors)
B = enforce_id2(dim, num_sensors)
C = sensor_constraints(dim, num_sensors)
D = anchor_constraints(num_anchors, num_sensors)

In [5]:
def sum_elem_product_noncvx(A,B):
    return np.sum(np.multiply(A,B))

def make_AX_less_b(X):
    output = []
    for id_constraint, rhs in zip(A[0], A[1]):
        output.append(sum_elem_product_noncvx(id_constraint, X) - rhs)
    for id_constraint2, rhs in zip(B[0], B[1]):
        output.append(sum_elem_product_noncvx(id_constraint2, X) - rhs)
    for sensor_constraint, rhs in zip(C[0], C[1]):
        output.append(sum_elem_product_noncvx(sensor_constraint, X) - rhs)
    for anchor_constraint, rhs in zip(D[0], D[1]):
        output.append(sum_elem_product_noncvx(anchor_constraint, X) - rhs)
    
    return output

def scriptAt_y(y):
    output = np.zeros((dim+num_sensors,dim+num_sensors)) #start sum of y_i*A_i
    i = 0
    for id_constraint, rhs in zip(A[0], A[1]):
        output += np.multiply(id_constraint, y[i])
        i = i + 1
    for id_constraint2, rhs in zip(B[0], B[1]):
        output += np.multiply(id_constraint2, y[i])
        i = i + 1
    for sensor_constraint, rhs in zip(C[0], C[1]):
        output += np.multiply(sensor_constraint, y[i])
        i = i + 1
    for anchor_constraint, rhs in zip(D[0], D[1]):
        output += np.multiply(anchor_constraint, y[i])
        i = i + 1
    return output

def grad(X):
    return scriptAt_y(make_AX_less_b(X))

In [6]:
def eigen_decomp_project(X_k1):
    [eigenvalues, V] = np.linalg.eig(X_k1)
    eigenvalues[eigenvalues < 0] = 0
    Lambda = np.diag(eigenvalues)
    return np.dot(np.dot(V, Lambda), np.transpose(V))

In [7]:
def ldl_decomp_project(X_k1):
    def ldl_decomp(A):
        A = np.matrix(A)

        S = np.diag(np.diag(A))
        Sinv = np.diag(1/np.diag(A))
        D = np.matrix(S.dot(S))
        Lch = np.linalg.cholesky(A)
        L = np.matrix(Lch.dot(Sinv))
        return L, D
    [L, D] = ldl_decomp(X_k1)
    D[D < 0] = 0
    D_psd = np.diag(D)
    return np.dot(np.dot(L, D), np.transpose(L))

In [14]:
def pca_proj(X):
    [evalues, evectors] = scipy.linalg.eigh(X)
    
    idx = evalues.argsort()[::-1]   
    evalues = evalues[idx]
    evectors = evectors[:,idx]
    
    matrix = np.zeros(X.shape)
    width = evalues.shape[0]
    for i in range(9):
        index = width - i - 1
        if index >= 0 and evalues[index] > 0:
            matrix += evalues[index] * np.outer(evectors[:, index], evectors[:, index])
    return matrix

In [9]:
Beta = 100
def run_solver(method_name, projection_func):
    thing = random.rand(dim + num_sensors, dim + num_sensors)
    X_0 = np.dot(thing,thing.transpose())
    X_k = X_0
    k = 0
    check = 1000
    max_iter = 20000
    
    while check > 10**-8 and k < max_iter:
        # Get next iterate from descent
        X_k1 = X_k - (1/Beta) * grad(X_k)

        # Project back into cone with eigen decomp
        X_k1 = projection_func(X_k1)

        #set up for next iteration
        check = np.linalg.norm(X_k1 - X_k)  
        X_k = X_k1

        k = k+1
    
    print("Real sensors: ")
    print(sensors)
    print("Generated Sensors for method " + method_name)
    generated = np.transpose(X_k[0:dim, dim:dim+num_sensors])
    print(generated)

In [10]:
run_solver("Eigenvalue Decomposition", eigen_decomp_project)

Real sensors: 
[[ 0.97591557 -1.90590207]
 [ 0.98879867 -0.61392471]
 [ 1.0582781  -1.50743397]
 [-1.61550811  0.52106587]
 [ 1.26337953  1.12418618]
 [ 1.10768273  1.44839876]
 [ 0.68032683  0.17353154]
 [ 1.07754533  1.49970902]
 [ 0.34527882 -0.13375661]
 [ 0.79291076 -1.78982923]]
Generated Sensors for method Eigenvalue Decomposition
[[ 0.9690596  -1.90176948]
 [ 0.98165871 -0.60973985]
 [ 1.05117266 -1.50322042]
 [-1.61638973  0.52271575]
 [ 1.25519359  1.12868242]
 [ 1.0997615   1.4527689 ]
 [ 0.67372273  0.17745107]
 [ 1.06967858  1.50405406]
 [ 0.33960666 -0.13018334]
 [ 0.78641769 -1.78584476]]


In [13]:
# Diverges with error because the matrix eventually becomes non positive-definite
# run_solver("LDL Decomposition", ldl_decomp_project)

In [15]:
run_solver("Top 6 Eigenavalues", pca_proj)

Generated Sensors for method Top 6 Eigenavalues
[[-0.08221924  0.18045344]
 [-0.02928192  0.03100771]
 [-0.40077771  0.20577432]
 [ 0.44870151 -0.11630888]
 [-0.06072766 -0.11967323]
 [-0.11978896 -0.04817487]
 [-0.07542121 -0.02036381]
 [ 0.00876687 -0.13781145]
 [ 0.07008961  0.02656377]
 [-0.09297427  0.12753467]]
